In [1]:
!pip install pycaret

     |████████████████████████████████| 188 kB 9.1 MB/s 
     |████████████████████████████████| 93 kB 939 kB/s 
     |████████████████████████████████| 7.3 MB 60.7 MB/s 
     |████████████████████████████████| 7.0 MB 43.3 MB/s 
     |████████████████████████████████| 259 kB 64.4 MB/s 
     |████████████████████████████████| 127 kB 45.9 MB/s 
     |████████████████████████████████| 3.1 MB 63.3 MB/s 
     |████████████████████████████████| 378 kB 46.2 MB/s 
     |████████████████████████████████| 81 kB 7.5 MB/s 
     |████████████████████████████████| 60 kB 5.5 MB/s 
     |████████████████████████████████| 63.9 MB 40.0 MB/s 
     |████████████████████████████████| 142.8 MB 38 kB/s 
     |████████████████████████████████| 2.1 MB 45.4 MB/s 
     |████████████████████████████████| 46 kB 2.8 MB/s 
     |████████████████████████████████| 6.2 MB 53.0 MB/s 
     |████████████████████████████████| 64 kB 2.2 MB/s 
     |████████████████████████████████| 235 kB 44.5 MB/s 
  Created wheel for pyod

In [2]:
from pycaret.regression import *
import pandas as pd

train = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv', index_col=0)
test = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv', index_col=0)
full_data = pd.concat([train.drop('SalePrice', axis=1), test])

train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
CATEGORICAL_FEATURES = [
    'LotConfig',
    'Neighborhood',
    'HouseStyle',
    'Exterior1st',
    'Exterior2nd',
    'Foundation',
    'has_garage',
    'has_pool',
]

# ORDINAL_FEATURES = {
#     'OverallQual': ['1','2','3','4','5','6','7','8','9','10'],
#     'OverallCond': ['1','2','3','4','5','6','7','8','9','10'],
# }

NUMERIC_FEATURES = [
    'LotArea',
    'BedroomAbvGr',
    'total_sf', # 1stFlrSF + 2ndFlrSF
    'bathrooms', # FullBath + (HalfBath * 0.5)
    'age',
    # 'remodel_age',
]

# ALL_FEATURES = CATEGORICAL_FEATURES + list(ORDINAL_FEATURES.keys()) + NUMERIC_FEATURES
ALL_FEATURES = CATEGORICAL_FEATURES + NUMERIC_FEATURES

In [4]:
def preprocess(df):
    df['has_garage'] = pd.notna(df['GarageType'])
    df['has_pool'] = pd.notna(df['PoolQC'])
    df['total_sf'] = df['1stFlrSF'] + df['2ndFlrSF']
    df['bathrooms'] = df['FullBath'] + (df['HalfBath'] * 0.5)
    df['age'] = 2020 - df['YearBuilt']
    return df[ALL_FEATURES]

p_train = preprocess(train).join(train.SalePrice)
p_test = preprocess(test)

In [5]:
env = setup(data = p_train,
            target='SalePrice', 
            categorical_features = CATEGORICAL_FEATURES,
            # ordinal_features = ORDINAL_FEATURES,
            numeric_features = NUMERIC_FEATURES,
            normalize = True,
            remove_outliers = True,
            silent = True)

 
Setup Succesfully Completed!


,Description,Value
0,session_id,5858
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(1460, 14)"
4,Missing Values,False
5,Numeric Features,5
6,Categorical Features,8
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [6]:
# compare_models(turbo=True) # easy pick: catboost

In [7]:
cb = create_model('catboost')
tuned_cb = tune_model('catboost', optimize='mae')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,23643.8963,2.274819e+09,47695.0651,0.8007,0.1772,0.1238
1,22179.7914,1.166813e+09,34158.6494,0.8347,0.1629,0.1196
2,19774.8570,6.738245e+08,25958.1304,0.8590,0.1472,0.1159
3,17837.1534,6.427760e+08,25353.0271,0.8601,0.1532,0.1122
4,17473.2131,5.632911e+08,23733.7542,0.8715,0.1673,0.1194
5,19065.7365,9.218488e+08,30361.9628,0.8761,0.1360,0.1026
6,18113.9184,5.515121e+08,23484.2941,0.8495,0.1420,0.1111
7,15829.8006,4.178084e+08,20440.3624,0.9286,0.1623,0.1186
8,18748.3797,6.249551e+08,24999.1017,0.8745,0.1610,0.1220
9,20733.4543,1.009709e+09,31775.9202,0.7437,0.1738,0.1243


In [8]:
predictions = predict_model(tuned_cb, data=p_test)
submission = pd.concat([pd.Series(p_test.index), predictions['Label']], axis=1)
submission.columns = ['Id', 'SalePrice']
submission.to_csv('submission.csv',index=False)
submission.head()

,Id,SalePrice
0,1461,126790.7923
1,1462,163127.8780
2,1463,195081.7041
3,1464,174895.4338
4,1465,198859.9059
